# Import dependencies

In [1]:
import numpy as np
import tensorflow as tf

import gensim.downloader as gensim_api
from nltk.tokenize import TreebankWordTokenizer

# Load word vectors

In [2]:
word_vectors = gensim_api.load("word2vec-google-news-300")  # 300-dimensional vectors

# Process text

In [3]:
def tokenize_and_vectorize(dataset_):
    tokenizer_ = TreebankWordTokenizer()
    vectorized_data_ = []
    expected_ = []
    for sample_ in dataset_:
        tokens_ = tokenizer_.tokenize(sample_[1])
        sample_vecs_ = []
        for token_ in tokens_:
            try:
                sample_vecs_.append(word_vectors[token_])
            except KeyError:
                pass
        vectorized_data_.append(sample_vecs_)
        expected_.append(sample_[0])
    return vectorized_data_, expected_

In [4]:
def pad_trunc(data_, maxlen_):
    new_data_ = []
    zero_vector_ = [0.0 for _ in range(len(data_[0][0]))]
    for sample_ in data_:
        if len(sample_) > maxlen_:
            temp_ = sample_[:maxlen_]
        elif len(sample_) < maxlen_:
            temp_ = sample_
            additional_elems_ = maxlen_ - len(sample_)
            for _ in range(additional_elems_):
                temp_.append(zero_vector_)
        else:
            temp_ = sample_
        new_data_.append(temp_)
    return new_data_

# Load the model

In [7]:
with open("resulting_models/CONV_sentimal/cnn_model.json", 'r') as json_file:
    json_string = json_file.read()
model = tf.keras.models.model_from_json(json_string)
model.load_weights('resulting_models/CONV_sentimal/cnn_weights.weights.h5')

# Do experiment

In [12]:
statement = {
    0: "NEGATIVE",
    1: "POSITIVE",
}

def is_positive(prediction_array):
    return statement[0] if prediction_array[0][0] < 0.5 else statement[1]

In [13]:
sample_1 = "I hate that the dismal weather had me down for so long, " \
           "when will it break! Ugh, when does happiness return? The sun is blinding " \
           "and the puffy clouds are too thin. I can't wait for the weekend."

maxlen = 400
embedding_dims = 300
dataset = [[1, sample_1]]
vec_list, expected = tokenize_and_vectorize(dataset)
test_vec_list = pad_trunc(vec_list, maxlen)
test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))
prediction = model.predict(test_vec)

print('Prediction for negative sentence:', is_positive(prediction))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Prediction for negative sentence: NEGATIVE


In [14]:
sample_1 = """Every day brings new opportunities to learn, grow, and make a difference. With a little effort and a positive mindset, even small steps can lead to incredible progress. Remember, each challenge we face is a chance to become stronger and more resilient. Celebrate the little wins along the way, and don't forget that you're capable of achieving amazing things. The future is bright, and each decision you make today builds towards a better tomorrow. Keep going—you've got this!"""

maxlen = 400
embedding_dims = 300
dataset = [[1, sample_1]]
vec_list, expected = tokenize_and_vectorize(dataset)
test_vec_list = pad_trunc(vec_list, maxlen)
test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))
prediction = model.predict(test_vec)

print('Prediction for negative sentence:', is_positive(prediction))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Prediction for negative sentence: POSITIVE


In [15]:
sample_1 = """Some days, no matter how much effort you put in, things just don’t seem to go right. Progress feels slow, and setbacks can feel overwhelming. It’s tough when challenges keep piling up and solutions seem out of reach. Frustrations build, and it can be hard to see a way forward. Even the small wins don’t seem to add up, leaving a sense that all efforts might be in vain."""

maxlen = 400
embedding_dims = 300
dataset = [[1, sample_1]]
vec_list, expected = tokenize_and_vectorize(dataset)
test_vec_list = pad_trunc(vec_list, maxlen)
test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))
prediction = model.predict(test_vec)

print('Prediction for negative sentence:', is_positive(prediction))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Prediction for negative sentence: NEGATIVE


In [16]:
# TODO pass more negative and positive samples